In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH16-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Evaluations


In [3]:
import pandas as pd

df = pd.read_csv("./data/ragas_synthetic_dataset.csv")
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What role does content moderation play in the ...,"['£유튜브, 특정인을 모방한 AI 생성 콘텐츠에 대한 삭제 요청에도 대응 계획\n...",Content moderation plays a crucial role in AI ...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
1,What are the key aspects of the AI Development...,"['SPRi AI Brief |\n2023-12월호\nG7, 히로시마 AI 프로세스...",The key aspects of the AI Development Guidelin...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
2,What was the purpose of the AI Safety Summit h...,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n영국 AI 안...,The purpose of the AI Safety Summit held in No...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
3,What does 'Operationalizing Progress' refer to...,"['구글 딥마인드, 범용 AI 모델의 기능과 동작에 대한 분류 체계 발표\nKEY ...",The term 'Operationalizing Progress' in the co...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
4,What improvements does Tongyi Qianwen 2.0 have...,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n알리바바 클라...,Tongyi Qianwen 2.0 has improvements over the p...,reasoning,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True


# DeepL을 활용한 번역

발급받은 키를 `.env`파일에 `DEEPL_API_KEY`로 저장

In [4]:
#!pip install -qU deel

In [5]:
import os

from langchain_teddynote.translate import Translator

deepl_api_key = os.getenv("DEEPL_API_KEY")

translator = Translator(deepl_api_key, "EN", "KO")

#번역 실행
translated_text = translator("hello, nice to meet you")
print(translated_text)

안녕하세요, 만나서 반가워요


In [6]:
from tqdm import tqdm
for i, row in tqdm(df.iterrows(), total=len(df), desc="번역진행중"):
    df.loc[i, "question_translated"] = translator(row["question"])
    df.loc[i, "ground_truth_translated"] = translator(row["ground_truth"])


번역진행중: 100%|██████████| 10/10 [00:11<00:00,  1.14s/it]


In [7]:
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done,question_translated,ground_truth_translated
0,What role does content moderation play in the ...,"['£유튜브, 특정인을 모방한 AI 생성 콘텐츠에 대한 삭제 요청에도 대응 계획\n...",Content moderation plays a crucial role in AI ...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,AI 개발의 맥락에서 콘텐츠 조정은 어떤 역할을 하나요?,콘텐츠 조정은 AI 시스템이 적절한 데이터로 학습하고 유해하거나 부적절한 콘텐츠가 ...
1,What are the key aspects of the AI Development...,"['SPRi AI Brief |\n2023-12월호\nG7, 히로시마 AI 프로세스...",The key aspects of the AI Development Guidelin...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,G7이 수립한 AI 개발 가이드라인의 주요 내용은 무엇인가요?,G7이 마련한 AI 개발 가이드라인의 주요 내용은 첨단 AI 시스템에 대한 국제 행...
2,What was the purpose of the AI Safety Summit h...,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n영국 AI 안...,The purpose of the AI Safety Summit held in No...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,2023년 11월에 개최된 AI 안전 서밋의 목적은 무엇인가요?,2023년 11월에 개최된 AI 안전 서밋의 목적은 다양한 분야에서의 AI의 역할에...
3,What does 'Operationalizing Progress' refer to...,"['구글 딥마인드, 범용 AI 모델의 기능과 동작에 대한 분류 체계 발표\nKEY ...",The term 'Operationalizing Progress' in the co...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,AGI 개발의 맥락에서 '운영화 진행'이란 무엇을 의미하나요?,AGI 개발의 맥락에서 '운영화 진행'이라는 용어는 AGI 기능의 발전을 구현하고 ...
4,What improvements does Tongyi Qianwen 2.0 have...,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n알리바바 클라...,Tongyi Qianwen 2.0 has improvements over the p...,reasoning,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,통이췐웬 2.0은 이전 버전에 비해 어떤 점이 개선되었나요?,"통이췐웬 2.0은 이전 버전에 비해 MMLU, GSM8k, ARC-C 등 다양한 벤..."


번역전

In [8]:
df.loc[:, ["question", "ground_truth"]]

,question,ground_truth
0,What role does content moderation play in the ...,Content moderation plays a crucial role in AI ...
1,What are the key aspects of the AI Development...,The key aspects of the AI Development Guidelin...
2,What was the purpose of the AI Safety Summit h...,The purpose of the AI Safety Summit held in No...
3,What does 'Operationalizing Progress' refer to...,The term 'Operationalizing Progress' in the co...
4,What improvements does Tongyi Qianwen 2.0 have...,Tongyi Qianwen 2.0 has improvements over the p...
5,What features of Claude LLM have advanced NLP ...,The answer to given question is not present in...
6,How does data integrity support transparency i...,The answer to given question is not present in...
7,What key AI events in 2024 will showcase the e...,The key AI events in 2024 that will showcase t...
8,What challenges come up in the EU's AI Act if ...,The challenges that arise in the EU's AI Act i...
9,What factors matter for ethical AI innovation?,The context discusses various factors that mat...


In [9]:
df.loc[:, ["question_translated", "ground_truth_translated"]]

,question_translated,ground_truth_translated
0,AI 개발의 맥락에서 콘텐츠 조정은 어떤 역할을 하나요?,콘텐츠 조정은 AI 시스템이 적절한 데이터로 학습하고 유해하거나 부적절한 콘텐츠가 ...
1,G7이 수립한 AI 개발 가이드라인의 주요 내용은 무엇인가요?,G7이 마련한 AI 개발 가이드라인의 주요 내용은 첨단 AI 시스템에 대한 국제 행...
2,2023년 11월에 개최된 AI 안전 서밋의 목적은 무엇인가요?,2023년 11월에 개최된 AI 안전 서밋의 목적은 다양한 분야에서의 AI의 역할에...
3,AGI 개발의 맥락에서 '운영화 진행'이란 무엇을 의미하나요?,AGI 개발의 맥락에서 '운영화 진행'이라는 용어는 AGI 기능의 발전을 구현하고 ...
4,통이췐웬 2.0은 이전 버전에 비해 어떤 점이 개선되었나요?,"통이췐웬 2.0은 이전 버전에 비해 MMLU, GSM8k, ARC-C 등 다양한 벤..."
5,Claude LLM의 어떤 기능이 NLP와 AI를 발전시켰나요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
6,데이터 무결성은 출처의 투명성을 어떻게 지원하나요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
7,2024년에 AI 기술의 경제적 영향을 보여줄 주요 AI 이벤트에는 어떤 것이 있을까요?,AI 기술의 경제적 영향을 보여줄 2024년의 주요 AI 행사로는 다양한 AI 기술...
8,EU의 AI 법에서 기초 모델을 우선시할 경우 어떤 문제가 발생할까요?,기초 모델이 우선순위가 될 경우 EU의 AI 법에서 발생하는 과제에는 기초 모델에 ...
9,윤리적 AI 혁신을 위해 중요한 요소는 무엇인가요?,"이 맥락에서는 AI 개발에서 책임감 있는 관행의 필요성, 투명성의 중요성, AI 시..."


In [10]:
# 번역된것을 question과 grund_truth로 치환해준다.

df.drop(columns=["question", "ground_truth"], inplace=True)
df.rename(
    columns={
        "question_transrated": "question",
        "ground_truth_translated": "ground_truth"
    },
    inplace=True
)
df.head()

,contexts,evolution_type,metadata,episode_done,question_translated,ground_truth
0,"['£유튜브, 특정인을 모방한 AI 생성 콘텐츠에 대한 삭제 요청에도 대응 계획\n...",simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,AI 개발의 맥락에서 콘텐츠 조정은 어떤 역할을 하나요?,콘텐츠 조정은 AI 시스템이 적절한 데이터로 학습하고 유해하거나 부적절한 콘텐츠가 ...
1,"['SPRi AI Brief |\n2023-12월호\nG7, 히로시마 AI 프로세스...",simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,G7이 수립한 AI 개발 가이드라인의 주요 내용은 무엇인가요?,G7이 마련한 AI 개발 가이드라인의 주요 내용은 첨단 AI 시스템에 대한 국제 행...
2,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n영국 AI 안...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,2023년 11월에 개최된 AI 안전 서밋의 목적은 무엇인가요?,2023년 11월에 개최된 AI 안전 서밋의 목적은 다양한 분야에서의 AI의 역할에...
3,"['구글 딥마인드, 범용 AI 모델의 기능과 동작에 대한 분류 체계 발표\nKEY ...",simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,AGI 개발의 맥락에서 '운영화 진행'이란 무엇을 의미하나요?,AGI 개발의 맥락에서 '운영화 진행'이라는 용어는 AGI 기능의 발전을 구현하고 ...
4,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n알리바바 클라...,reasoning,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,통이췐웬 2.0은 이전 버전에 비해 어떤 점이 개선되었나요?,"통이췐웬 2.0은 이전 버전에 비해 MMLU, GSM8k, ARC-C 등 다양한 벤..."


In [11]:
# 번역한 데이터셋 저장
df.to_csv("data/ragas_synthetic_dataset_translated.csv", index=False)

# Hugging Face Dataset 업로드

In [12]:
# 데이터셋으로 변환
import os
from datasets import Dataset


dataset = Dataset.from_pandas(df)

#데이터셋 이름 정의. 허깅페이스 데이터셋에서 만들어줘야한다.
dataset_name = "Jaehoya/ragas-test-dataset"

# 데이터셋 업로드
dataset.push_to_hub(
    dataset_name,
    private=True,  # private=False로 설정하면 공개 데이터셋이 됩니다.
    split="korean_v1",  # 데이터셋 split 이름 입력
    token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
pd.DataFrame(dataset)[:3]

,contexts,evolution_type,metadata,episode_done,question_translated,ground_truth
0,"['£유튜브, 특정인을 모방한 AI 생성 콘텐츠에 대한 삭제 요청에도 대응 계획\n...",simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,AI 개발의 맥락에서 콘텐츠 조정은 어떤 역할을 하나요?,콘텐츠 조정은 AI 시스템이 적절한 데이터로 학습하고 유해하거나 부적절한 콘텐츠가 ...
1,"['SPRi AI Brief |\n2023-12월호\nG7, 히로시마 AI 프로세스...",simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,G7이 수립한 AI 개발 가이드라인의 주요 내용은 무엇인가요?,G7이 마련한 AI 개발 가이드라인의 주요 내용은 첨단 AI 시스템에 대한 국제 행...
2,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n영국 AI 안...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,2023년 11월에 개최된 AI 안전 서밋의 목적은 무엇인가요?,2023년 11월에 개최된 AI 안전 서밋의 목적은 다양한 분야에서의 AI의 역할에...
